In [1]:
import datetime
import ujson
import numpy 
data = ujson.load(open('user_project_times.json'))

In [2]:
summaries = {}
ctr = 0
for line in open('user_project_summaries.json'): # lazy iteration because the file is large
    print ctr,
    ctr+=1
    summaries.update(ujson.loads(line))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46


In [3]:
TASK = "LANGUAGE"

Create Language Data Set and Initialize Language Codes

In [4]:
'''We are using all the data from the user from whom we we have previously classfied what their primary langauge is '''
user_langs = ujson.load(open('user_inferredlangs.json')) 
lang_data = {}
keys = user_langs.keys()

print 'done loading keys'
for key in keys: 
    lang_data[key] = data[key[0:2] + '/' + key]
print 'lang data created'

lang_dict = {} #key is langauge isocode, value is number 
isocodes = ujson.load(open('isocodes.json'))
counter = 0 
for lang in isocodes:
    lang_dict[lang] = counter
    counter+=1
    

done loading keys
lang data created


In [5]:
def convert_time(el):
    el = int(str(el)[:10])
    return datetime.date.fromtimestamp(el)

Now let's get the length of activity for each user (still using creation times) in days. 

In [6]:
import numpy
user_activity_length = numpy.zeros(len(data))
for i, user in enumerate(data.keys()):
    ctimes = [ctime for ctime, _ in data[user]]
    user_activity_length[i] = (max(ctimes) - min(ctimes))/(86400*1000)

The histogram shows the number of users whose activity time was within `x` number of days.

In [7]:
earliest = 1e100
latest = 0
for user in data:
    for ctime, _ in data[user]: #ignore last-modified for now
        if ctime < earliest:
            earliest = ctime
        if ctime > latest:
            latest = ctime

print 'Earliest:', convert_time(earliest) 
print 'Latest:', convert_time(latest)
MAXDUR = 300
MINDUR = 150
# get subset of data from "old" users whose earliest creation date is more than MAXDUR days before the end of the dataset
# (giving them a good chance to stay on for more than MINDUR days),
# AND have activity within MAXDUR days if they are active beyond MINDUR (to be fair to recent users)
old_data = {}
for user in data:
    start_time = min([ctime for ctime, _ in data[user]])
    if int((latest-start_time)/(86400.*1000))>MAXDUR:
        post = filter(lambda length: length>MINDUR, 
                      map(lambda (ctime,mtime): (ctime-start_time)/(86400.*1000), data[user]))
        if len(post)>0 and len(filter(lambda length: length<=MAXDUR, post))==0:
            continue  # ignore users who have no activity in the 120-180 period
        old_data[user[3:]] = summaries[user[3:]]
print 'Filtered to', len(old_data), 'users from', len(data)

old_user_activity_length = numpy.zeros(len(old_data))
for i, user in enumerate(old_data.keys()):
    ctimes = [ctime for ctime, _ in data[user[0:2] + '/' + user]]
    old_user_activity_length[i] = (max(ctimes) - min(ctimes))/(86400*1000)

Earliest: 2013-03-27
Latest: 2016-03-10
Filtered to 25324 users from 46319


In [39]:
#from features import *



In [8]:
if TASK == 'RETENTION': 
    data = old_data
if TASK == 'LANGUAGE':
    data = lang_data

In [9]:
def getEarliest(userID):
    return data[userID][0][0]

In [33]:

print getEarliest(keys[0])

1443070339416


In [10]:
def getEarliestSummary(userID): #helper function to get the earliest project creation date for summmaries
    user = summaries[userID]
    earliest = user.keys()[0]
    for proj in user: 
        creationDate = summaries[userID][proj]['**created']
        if  creationDate < earliest: 
            earliest = creationDate
    return earliest
        

In [11]:
keys = lang_data.keys()
# print keys[0:100]
# i = 0 
# for userID in keys: 
#     if i % 1000 == 0: 
#         print i 
#     getEarliestSummary(userID)
#     i += 1

In [19]:
users = lang_data.keys()[0:100]
print 'loaded keys'
for userID in users: 
    getEarliest(userID)

loaded keys


In [12]:
def getProjectsInMINDUR(userID): #for summaries
    projList = []
    earliest = getEarliest(userID)

    latest = (MINDUR * 86400 * 1000) + earliest 
    user = summaries[userID]
    for proj in user: 
        if summaries[userID][proj]['**created'] >= earliest and summaries[userID][proj]['**created'] <= latest: 
            projList.append(summaries[userID][proj])
    return projList
                        
                                                                                 

In [62]:
def getProjects(userID):
    if TASK == 'LANGUAGE': 
        return {userID: summaries[userID]} 
    if TASK == 'RETENTION':
        return {userID : getProjectsInMINDUR(userID)}

In [90]:
def sortedProjects(userID): 
    r = getProjects(userID)
    projects = r[r.keys()[0]] 
    #print projects[projects.keys()[0]]['**created']

    name_time = [(project, projects[project]['**created']) for project in projects]
    name_time = sorted(name_time, key=lambda t: t[1])  
    proj_list = [summaries[userID][project[0]] for project in name_time]
    return proj_list
    #print proj_list


100.0
100.0
86.3636363636
100.0
96.1538461538
95.4545454545
75.0
95.652173913
80.0
92.5925925926


In [112]:
def getAverageProjectLen(projects):
    num_projects = len(projects)
    total_project_length = 0 #the sum of all the lengths of the project (from date created to date modified)
    
    prev = projects[0]["**created"]
    for project in projects[1:]: 
        curr = project['**created']
        total_project_length += (curr - prev)/(86400.*1000)   #SR: added this to scale to days
        prev = curr 
    return int(total_project_length / float(num_projects))

In [122]:
def numProjects(projects):
    return len(projects)

In [123]:
def percentOnWeekday(projects):
    num_projs = numProjects(projects)
    weekday_projs = 0
    for project in projects: 
        if datetime.datetime.fromtimestamp(project['**created'] / 1e3).weekday() < 6: 
            weekday_projs += 1.0
    return weekday_projs / num_projs * 100

In [135]:
import datetime
def percentOnDay(projects):
    num_projs = len(projects)
    weekday_projs = 0
    dow = [] 
    for i in range(7):
        dow.append(0)
    for project in projects: 
        date = datetime.datetime.fromtimestamp(project['**created'] / 1e3).weekday() 
        dow[date] += 1.0
    for i in range(7):
        dow[i] = dow[i] / num_projs * 100 
    return dow

In [140]:
def getDecileBoundaries(projects):
    first= projects[len(projects)-1]['**created']
    decile_width = MINDUR * 86400000/ 10 
    decileNums = []
    for i in range(1,11): 
        decileNums.append(first + decile_width*i)
    return decileNums

In [205]:
 def numProjectsInDecile(projects):
    ''' the first decile is the 0th decile (covering from 0-10% of the time), 
    the last decile is the 9th decile'''
    first= int(projects[0]['**created'])
    #print first
    last = int(projects[len(projects)-1]['**created'])
    #print last
    decile_width = (last - first) / (86400 * 1000)
    decileList = []
    
    for i in range(10): 
        decileList.append(0)
        
    for project in projects: 
        if project['**created'] >= first and project['**created'] <= last: 
            index  = ((project['**created']-first)/(86400 * 10000))
            if index >= 10: 
                decileList[9] = decileList[9] + 1
            else: 
                decileList[index] = decileList[index] + 1
    return decileList
    

In [228]:
users = old_data.keys()[0:1]
for userID in users:
    r = sortedProjects(userID)


{u'**modified': 1394175531773L, u'**created': 1394173993464L, u'**Project Name': u'PlayMusic', u'*Media Assets': [u'skycity.mid'], u'Screen1': {u'Blocks': {u'Orphan Blocks': u'NO ORPHAN BLOCKS', u'*Top Level Blocks': {u'Slider.PositionChanged': 1, u'Button.Click': 3, u'global_declaration': 1}, u'Active Blocks': {u'Global Variable Names': {u'thumbPosition': 1, u'global vMusic': 4, u'vMusic': 1}, u'*Number of Blocks': 35, u'Procedure Names': {}, u'Local Variable Names': {}, u'Procedure Parameter Names': {}, u'Strings': {u'Basket.mid': 1, u'SkyCity.mid': 1, u'null': 1, u'Toto.mid': 1, u'\u6b63\u5728\u64ad\u653e ': 1}, u'Types': {u'Player.Stop': 1, u'Slider.PositionChanged': 1, u'lexical_variable_set': 3, u'CheckBox.GetChecked': 3, u'Player.SetSource': 1, u'Button.Click': 3, u'Player.Pause': 1, u'text': 5, u'controls_if': 1, u'Slider.GetThumbPosition': 1, u'Player.Start': 1, u'Label.SetText': 1, u'text_join': 1, u'Player.GetSource': 1, u'global_declaration': 1, u'Slider.SetThumbPosition': 

In [ ]:
#Summary features 

In [60]:
print summaries['44884'].keys()#['p021_020_clock'].keys()#['Screen1']['Blocks']['Active Blocks'].keys()#['Local Variable Names'].keys() #['Local Variable Names']

[u'p010_009_simplecalc', u'p021_020_clock', u'p012_011_molemash', u'p006_005_talktome', u'p018_018_webviewer', u'p005_008_lightmeup', u'p004_003_triplebutton', u'p017_017_spinner', u'p003_004_sound', u'p000_000_Helpme', u'p013_016_paintpot', u'p020_021_finalsmartinez', u'p002_002_topperformer', u'p014_012_mutiplescreens', u'p009_010_checkbox', u'p001_001_buttonimg', u'p008_006_timepicker', u'p011_014_ballbounce', u'p016_015_slider', u'p015_013_magic8ball', u'p019_019_coloreddots', u'p007_007_datepicker']


In [247]:
def getAveNumLocalVariables(projects):
    numScreens = getNumScreens(projects)
    lv_count = 0 
    if not noAbnormalities(userID, projName): return 0 
    for projectScreens in range(len(numScreens)):
        for i in range(projectScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][projName].keys():
                if summaries[userID][projName][screenNum]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                    lv_count += len(summaries[userID][projName][screenNum]['Blocks']['Active Blocks']['Local Variable Names'].keys())
    return lv_count

In [279]:
def getAveNumLocalVariables(projects):
    numScreens = getNumScreens(projects)
    #print numScreens
    #print projects
    lv_count = 0
    for i in range(len(projects)): 
        project = projects[i]
        for n in range(numScreens[i]):
            screenNum = "Screen"+ str(n + 1)
            if screenNum in project.keys():                    
                      if project[screenNum]["Blocks"]["Active Blocks"] != 'NO ACTIVE BLOCKS':
                        try: 
                            lv_count += len(project['Blocks']['Active Blocks']['Local Variable Names'].keys())
                        except KeyError: 
                            print "ERROR", project
    return lv_count / float(len(projects))

In [280]:
users = old_data.keys()[0:1]
for userID in users:
    r = sortedProjects(userID)
    print getAveNumLocalVariables(r)


ERROR {u'**modified': 1394175531773L, u'**created': 1394173993464L, u'**Project Name': u'PlayMusic', u'*Media Assets': [u'skycity.mid'], u'Screen1': {u'Blocks': {u'Orphan Blocks': u'NO ORPHAN BLOCKS', u'*Top Level Blocks': {u'Slider.PositionChanged': 1, u'Button.Click': 3, u'global_declaration': 1}, u'Active Blocks': {u'Global Variable Names': {u'thumbPosition': 1, u'global vMusic': 4, u'vMusic': 1}, u'*Number of Blocks': 35, u'Procedure Names': {}, u'Local Variable Names': {}, u'Procedure Parameter Names': {}, u'Strings': {u'Basket.mid': 1, u'SkyCity.mid': 1, u'null': 1, u'Toto.mid': 1, u'\u6b63\u5728\u64ad\u653e ': 1}, u'Types': {u'Player.Stop': 1, u'Slider.PositionChanged': 1, u'lexical_variable_set': 3, u'CheckBox.GetChecked': 3, u'Player.SetSource': 1, u'Button.Click': 3, u'Player.Pause': 1, u'text': 5, u'controls_if': 1, u'Slider.GetThumbPosition': 1, u'Player.Start': 1, u'Label.SetText': 1, u'text_join': 1, u'Player.GetSource': 1, u'global_declaration': 1, u'Slider.SetThumbPosit

In [ ]:
def getNumGlobalVariables(userID, projName):
    numScreens = getNumScreens(userID, projName)
    gv_count = 0 
    if not noAbnormalities(userID, projName): return 0 
    for i in range(numScreens):
        screenNum = "Screen"+ str(i + 1)

        if screenNum in summaries[userID][proj].keys():
            if summaries[userID][projName][screenNum]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                    gv_count += len(summaries[userID][projName][screenNum]['Blocks']['Active Blocks']['Global Variable Names'].keys())
    return gv_count

In [ ]:
def getAllVariables(userID, projName): 
    return getNumLocalVariables(userID, projName) + getNumGlobalVariables(userID, projName)

In [232]:
def averageLocalVar(projects):
    lv_count = 0
    
    for project in projects:
        lv_count += getNumLocalVariables(userID, project['**Project Name'])
    return lv_count/ float(len(projList))


In [ ]:
def averageGlobalVar(userID):
    projList = getProjectsNamesInMINDUR(userID)
    gv_count = 0 
    for projName in projList:
        gv_count += getNumGlobalVariables(userID, projName)
    return gv_count/ float(len(projList))


In [ ]:
def averageVar(userID): 
    projList = getProjectsNamesInMINDUR(userID)
    var_count = 0 
    for projName in projList:
        var_count += getAllVariables(userID, projName)
    return var_count/ float(len(projList))


In [ ]:
# keys = lang_data.keys()
# for userID in keys:
#     #print userID
#     projList = getProjectsNamesInMINDUR(userID)

#     for proj in projList:
#         getNumLocalVariables(userID,proj)
#         getNumGlobalVariables(userID, proj)
# #     print averageVar(userID)

In [214]:
def noAbnormalities(userID,projName): 
    numScreens = getNumScreens(userID, projName)

    for i in range(numScreens):
        screenNum = "Screen"+ str(i + 1)
        if screenNum not in summaries[userID][projName].keys():
            return False
        if summaries[userID][projName][screenNum]['Blocks']=='NO BLOCKS':
            return False
        blocks = summaries[userID][projName][screenNum]['Blocks']
        if blocks == 'MALFORMED BKYFILE':
            return False
        if blocks['Active Blocks']=='NO ACTIVE BLOCKS':
            return False
        if summaries[userID][projName][screenNum]['Components']=='N':
            return False
    return True

         

In [ ]:
# keys = summaries.keys()[0:100]

# for userID in keys:
#     projList = getProjectsNamesInMINDUR(userID)
#     for proj in projList:  
#         print noAbnormalities(userID, proj)
    
# for userID in keys: 
#     projList = getProjectsNamesInMINDUR(userID)
#     for proj in projList:
#         try: 
#             getNumLocalVariables(userID, proj)
#         except TypeError: 
#              print userID

In [243]:
def getNumScreens(projects):
    numScreens = []
    for project in projects: 
         numScreens.append(project['*Number of Screens'])
    return numScreens
    

In [ ]:
def averageNumScreens(userID):
    totalScreens = 0
    numProjs = 0
    validProjects = getProjectsNamesInMINDUR(userID)
    for project in validProjects: 
        numProjs += 1
        #print summaries[userID][project]
        totalScreens += getNumScreens(userID, project)
        
    return totalScreens/float(numProjs)
        

In [ ]:
def averageNumBlocks(userID):
    totalBlocks = 0 
    numProjs = 0
    validProjects = getProjectsNamesInMINDUR(userID)
    if validProjects ==0: return 0
    for proj in validProjects: 
        numScreens = getNumScreens(userID, proj)
        numProjs += 1
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys(): 
                if noAbnormalities(userID): 
                    totalBlocks += summaries[userID][proj][screenNum]['Blocks']['Active Blocks']['*Number of Blocks']
                else: return 0
    return totalBlocks/ float(numProjs)


In [ ]:
for userID in old_data: 
    try: 
        averageNumBlocks(userID)
    except TypeError:
        print userID

In [ ]:
def averageOrphanBlocks(userID): 
    numProjs = 0
    oBlocks = 0
    validProjects = getProjectsNamesInMINDUR(userID)
    for proj in validProjects: 
        numScreens = getNumScreens(userID, proj)
        numProjs += 1
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and 'Blocks' in summaries[userID][proj][screenNum].keys() and 'Orphan Blocks' in summaries[userID][proj][screenNum]['Blocks']: #occassional error otherwise
                if (summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks'] != 'NO ORPHAN BLOCKS'): 
                    oBlocks += summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
    return oBlocks / float(numProjs)
    
                    

In [ ]:
def averageNumTopLevelBlocks(userID): #per project, maybe do another function averageNumTopLevelBlocks per screen
    validProjects = getProjectsNamesInMINDUR(userID)
    totalTL = 0 
    numProjs = 0
    for proj in validProjects: 
        numScreens = getNumScreens(userID, proj)
        numProjs += 1
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and '*Top Level Blocks' in summaries[userID][proj][screenNum]['Blocks']: 
                typesOfTL = summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'].keys()
                for block in typesOfTL: 
                    totalTL += summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'][block]
    return totalTL/float(numProjs)
    
                
            
    

In [ ]:
def getDecileBoundaries(userID):
    earliest = getEarliest(userID)
    #print (earliest + MINDUR * 86400000)

    decile_width = MINDUR * 86400000/ 10 
    #print decile_width
    decileNums = []
    for i in range(1,11): 
        decileNums.append(earliest + decile_width*i)
    return decileNums

In [ ]:
def decileNumScreens(userID): 
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    
    for proj in projs: 
        date = summaries[userID][proj]['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                numScreens = summaries[userID][proj]['*Number of Screens']
                all_deciles[index].insert(0,numScreens)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles

In [ ]:
def decileNumScreens2(userID): 
    all_deciles = []
    earliest = int(getEarliest(userID))
    #print earliest
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    for proj in projs: 
        date = summaries[userID][proj]['**created']
        index  = ((date-earliest)/(86400 * 1000))/(MINDUR/10)
        #print index
        all_deciles[index].insert(0,getNumScreens(userID,proj))
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles

In [ ]:
# keys =  summaries.keys()
# for userID in keys:
#     if userID != '00999':
#         decileNumScreens2(userID)
#         decileNumTopLevelBlocks2(userID)
#         decileOrphanBlocks2(userID)

In [ ]:
def decileNumTopLevelBlocks2(userID): 
    '''measures how many TLblocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    
    for proj in projs: 
        totalTL =0 
        numScreens = getNumScreens(userID, proj)
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and '*Top Level Blocks' in summaries[userID][proj][screenNum]['Blocks']: 
                typesOfTL = summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'].keys()
                for block in typesOfTL: 
                    totalTL += summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'][block]
        date = summaries[userID][proj]['**created']
        index  = ((date-earliest)/(86400 * 10000))/(MINDUR/10)
        #print index
        all_deciles[index].append(totalTL)
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles
    

In [ ]:
def decileNumTopLevelBlocks(userID): 
    '''measures how many TLblocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    
    for proj in projs: 
        totalTL =0 
        numScreens = getNumScreens(userID, proj)
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and '*Top Level Blocks' in summaries[userID][proj][screenNum]['Blocks']: 
                typesOfTL = summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'].keys()
                for block in typesOfTL: 
                    totalTL += summaries[userID][proj][screenNum]['Blocks']['*Top Level Blocks'][block]
        date = summaries[userID][proj]['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                all_deciles[index].insert(0,totalTL)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles
    

In [ ]:
def decileOrphanBlocks2(userID): 
    '''measures how many orphan blocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    
    for proj in projs: 
        oBlocks =0 
        numScreens = getNumScreens(userID, proj)
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and 'Blocks' in summaries[userID][proj][screenNum].keys() and 'Orphan Blocks' in summaries[userID][proj][screenNum]['Blocks']: #occassional error otherwise
                if summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks'] != "NO ORPHAN BLOCKS":
                    try: 
                        oBlocks += summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
                    except TypeError: 
                        print summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
        date = summaries[userID][proj]['**created']
        index  = ((date-earliest)/(86400 * 10000))/(MINDUR/10)
        #print index
        all_deciles[index].insert(0,oBlocks)
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles

In [ ]:
def decileOrphanBlocks(userID): 
    '''measures how many orphan blocks per project in each decile '''
    all_deciles = []
    for i in range(10):
         all_deciles.append([])
    projs = getProjectsNamesInMINDUR(userID)
    decile_boundaries = getDecileBoundaries(userID)
    
    for proj in projs: 
        oBlocks =0 
        numScreens = getNumScreens(userID, proj)
        for i in range(numScreens):
            screenNum = "Screen"+ str(i + 1)
            if screenNum in summaries[userID][proj].keys() and 'Blocks' in summaries[userID][proj][screenNum].keys() and 'Orphan Blocks' in summaries[userID][proj][screenNum]['Blocks']: #occassional error otherwise
                if summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks'] != "NO ORPHAN BLOCKS":
                    try: 
                        oBlocks += summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
                    except TypeError: 
                        print summaries[userID][proj][screenNum]['Blocks']['Orphan Blocks']['*Number of Blocks']
        date = summaries[userID][proj]['**created']
        placed = False
        for boundary in decile_boundaries: 
            if date < boundary and not placed: 
                index = decile_boundaries.index(boundary)
                all_deciles[index].insert(0,oBlocks)
                placed = True
    for i in range(10):
        if len(all_deciles[i]) == 0: 
            all_deciles[i] = 0
        else: 
            all_deciles[i] = sum(all_deciles[i])/ float(len(all_deciles[i]))
    return all_deciles

In [ ]:
# keys = summaries.keys()[0:100]
# for userID in keys: 
#     print decileOrphanBlocks(userID)

In [ ]:
#summaries['44884']['p021_020_clock']['**created']
#for userID in summaries.keys()[0:10]:
#     print userID
#     print averageNumScreens(userID)
#     print averageNumBlocks(userID)
#     print averageOrphanBlocks(userID)
#     print averageNumTopLevelBlocks(userID)
#      print decileNumScreens(userID
   # print decileOrphanBlocks(userID)

In [ ]:
#returns the number of projects a user started in a particular decile... doesn't always get all projects :(


In [ ]:
from sklearn.feature_extraction import DictVectorizer

def percentProjectsFeatures(userID):
    """return a dictionary mapping each time slot (10%, 20%, etc) to the percentage of projects created
    at that time,
    for a particular user represented as list of (creation times, modified times),
    using only their first MINDUR days of activity"""   
    userDict = {} 
    #note: feature names are for our reference only
    userDict["average length"] = getAverageProjectLen(userID)
    userDict["ten percent"] =  percentProjects(userID, 10)
    userDict["twenty percent"] = percentProjects(userID, 20)
    userDict["thirty percent"] =  percentProjects(userID, 30)
    userDict["forty percent"] = percentProjects(userID, 40)
    userDict["fifty percent"] =  percentProjects(userID, 50)
    userDict["sixty percent"] = percentProjects(userID, 60)
    userDict["seventy percent"] =  percentProjects(userID, 70)
    userDict["eighty percent"] = percentProjects(userID, 80)
    userDict["ninety percent"] =  percentProjects(userID, 90)
    userDict["hundred percent"] = percentProjects(userID, 100)
    return userDict


In [ ]:
def dayAnalysisFeatures(userID): 
    userDict = {} 
    userDict["weekday"] = percentOnWeekday(userID) #
    userDict["Monday"] = percentOnDay(userID, 0)
    userDict["Tuesday"] = percentOnDay(userID, 1)
    userDict["Wednesday"] = percentOnDay(userID, 2)
    userDict["Thursday"] = percentOnDay(userID, 3)
    userDict["Friday"] = percentOnDay(userID, 4)
    userDict["Saturday"] = percentOnDay(userID, 5)
    userDict["Sunday"] = percentOnDay(userID, 6)

    return userDict

In [ ]:
def decileFeatures(userID): 
    userDict = {} 
    userDict["1st"] = numProjectsInDecile(userID, 0)
    userDict["2nd"] = numProjectsInDecile(userID, 1)
    userDict["3rd"] = numProjectsInDecile(userID, 2)
    userDict["4th"] = numProjectsInDecile(userID, 3)
    userDict["5th"] = numProjectsInDecile(userID, 4)
    userDict["6th"] = numProjectsInDecile(userID, 5)
    userDict["7th"] = numProjectsInDecile(userID, 6)
    userDict["8th"] = numProjectsInDecile(userID, 7)
    userDict["9th"] = numProjectsInDecile(userID, 8)
    
    return userDict

In [ ]:
def summaryNumScreensDecile(userID): 
    userDict = {}
    decileNumScreen = decileNumScreens(userID)
    userDict["NS 1"] = decileNumScreen[0]
    userDict["NS 2"] = decileNumScreen[1]
    userDict["NS 3"] = decileNumScreen[2]
    userDict["NS 4"] = decileNumScreen[3]
    userDict["NS 5"] = decileNumScreen[4]
    userDict["NS 6"] = decileNumScreen[5]
    userDict["NS 7"] = decileNumScreen[6]
    userDict["NS 8"] = decileNumScreen[7]
    userDict["NS 9"] = decileNumScreen[8]
    userDict["NS 10"] = decileNumScreen[9]
    
 
    
    
    
    
    return userDict

In [ ]:
def summaryOBlockDecile(userID): 
    userDict = {}
    decileOrphanBlock = decileOrphanBlocks(userID)
    userDict["O 1"] = decileOrphanBlock[0]
    userDict["O 2"] = decileOrphanBlock[1]
    userDict["O 3"] = decileOrphanBlock[2]
    userDict["O 4"] = decileOrphanBlock[3]
    userDict["O 5"] = decileOrphanBlock[4]
    userDict["O 6"] = decileOrphanBlock[5]
    userDict["O 7"] = decileOrphanBlock[6]
    userDict["O 8"] = decileOrphanBlock[7]
    userDict["O 9"] = decileOrphanBlock[8]
    userDict["O 10"] = decileOrphanBlock[9]
    return userDict

In [ ]:
def summaryDecileTLBlocks(userID): 
    userDict = {}
    decileNumTopLevelBlock = decileNumTopLevelBlocks(userID)
    userDict["TL 1"] = decileNumTopLevelBlock[0]
    userDict["TL 2"] = decileNumTopLevelBlock[1]
    userDict["TL 3"] = decileNumTopLevelBlock[2]
    userDict["TL 4"] = decileNumTopLevelBlock[3]
    userDict["TL 5"] = decileNumTopLevelBlock[4]
    userDict["TL 6"] = decileNumTopLevelBlock[5]
    userDict["TL 7"] = decileNumTopLevelBlock[6]
    userDict["TL 8"] = decileNumTopLevelBlock[7]
    userDict["TL 9"] = decileNumTopLevelBlock[8]
    userDict["TL 10"] = decileNumTopLevelBlock[9]
    return userDict

In [ ]:
def summaryAverages(userID):
    userDict = {} 
    userDict["NS"] = averageNumScreens(userID)
    #userDict["NB"] = averageNumBlocks(userID)
    userDict["OB"] = averageOrphanBlocks(userID)
    return userDict

In [ ]:
# users_percent_dicts = map(summaryAverages, old_data.keys())
# vec = DictVectorizer()
# X = vec.fit_transform(users_percent_dicts) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html

Write other feature functions -- perhaps different interval sizes (instead of 10%), proportion of projects rather than number, etc. With some effort, we can get a measure of activity on weekends/evenings as features too. Experiment...

Give each feature function a distinct name, and re-run the above two cells with the function in place of `featurize` to see if the contingency tables change

Remember from the CS111 assignment that we can combine a list of feature functions to give a new feature function, so you should experiment with combinations as well.

In [ ]:
def combine_featfuncs(funclist):
    def combined(user):
        basedict = funclist[0](user)
        for f in funclist[1:]:
            basedict.update(f(user))
        return basedict
    return combined

RETENTION:

In [ ]:
from sklearn.preprocessing import normalize
user_dicts = []
# i = 0

if TASK == "RETENTION":
    for userID in old_data: #user is just the user's id num 
        combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, decileFeatures]) #,summaryNumScreensDecile,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) #a function object is being returned here?? 
        user_dicts.append(combined(userID))


    print ('done loading featurizers')
    

    vec = DictVectorizer()
    X = vec.fit_transform(user_dicts) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html

    X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 

    print X.shape


**Attempt 1: Clustering.** Is there any correlation between the clusters produced by these features, and whether or not a user is retained?

In [ ]:
from sklearn.cluster import KMeans
if TASK == 'RETENTION':

    KMeans_model = KMeans(n_clusters=2)
    users_clusterids = KMeans_model.fit_predict(X)

    from scipy.stats import chi2_contingency
    conting = numpy.zeros((2, 2))  # row = cluster ID (0, 1), col = #days (0 for <=120, 1 for >120)

    for i in range(len(users_clusterids)):
        cluster = users_clusterids[i]
        if old_user_activity_length[i]<=MINDUR:
            conting[cluster, 0] += 1
        else:
            conting[cluster, 1] += 1

# visualize
    print conting

# compute chi2 statistic to get the probability that there is no association between
# the clustering and whether or not the user is active for >120 days
# lower probabilities are better
    _, p, _, _ = chi2_contingency(conting)
    print 'There is a', p, 'probability that the clusters and retention are independent.'

**Attempt 2: Classification.**

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn import linear_model

if TASK=='RETENTION':
    y = numpy.array(map(lambda d: int(d>MINDUR), old_user_activity_length))  # labels (0 for less than MINDUR, 1 otherwise)

    percentpos = sum(y)/float(len(y))
    chance = max(percentpos, 1-percentpos)
    print 'Chance is', chance

    print 'Building a model with', X.shape[1], 'features for', TASK
    all_coefs = [] 
    for trainidx, testidx in StratifiedKFold(y):
        print 'Fold'
        k = 70 #num neighbors
        Xtrain = X[trainidx, :]  # using numpy's smart indexing
        Xtest = X[testidx, :]
        ytrain = y[trainidx]
        ytest = y[testidx]

    
        KNeighbors_model = KNeighborsClassifier(n_neighbors=k)  
        logistic_model = linear_model.LogisticRegression(fit_intercept= False)
    
        KNeighbors_model.fit(Xtrain, ytrain)
        logistic_model.fit(Xtrain, ytrain)

        KNN_XtestP = KNeighbors_model.predict(Xtest)
        log_XtestP = logistic_model.predict(Xtest)
    

    
        print k, 'neighbors' 
        print "KNN Score: ", KNeighbors_model.score(Xtest, ytest)
        print 'KNN f1 score', f1_score(KNN_XtestP, ytest),'\n'

        print'Logistic Regression Score: ', logistic_model.score(Xtest, ytest)
        print 'Logistic f1 score', f1_score(log_XtestP, ytest) 
        print "max",numpy.amax(logistic_model.coef_)
        print "min", numpy.amin(logistic_model.coef_)
        print'Coefficients:', logistic_model.coef_
    



LANGUAGE:
    

In [ ]:
from sklearn.preprocessing import normalize
user_dicts = []

for userID in lang_data: #user is just the user's id num 
    combined = combine_featfuncs([percentProjectsFeatures, dayAnalysisFeatures, decileFeatures]) #,summaryNumScreensDecile,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages]) #a function object is being returned here?? 
    user_dicts.append(combined(userID))


print ('done loading featurizers')
    

vec = DictVectorizer()
X = vec.fit_transform(user_dicts) # converts to array: http://scikit-learn.org/stable/modules/feature_extraction.html

#X = normalize(X, axis=0, norm='l1') #normalization is bringing the values down significantly 

print X.shape


In [ ]:
# print len(data)
# print len(lang_data)
# #what y is
# print [lang_dict[user_langs[userID]] for userID in lang_data]

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 

print old_user_activity_length[0:100]
if TASK=='LANGUAGE':
    y =  numpy.array([lang_dict[user_langs[userID]] for userID in lang_data]) # labels

percentpos = sum(y)/float(len(y))
chance = max(percentpos, 1-percentpos)
print 'Chance is', chance

print 'Building a model with', X.shape[1], 'features for', TASK
all_coefs = [] 
for trainidx, testidx in StratifiedKFold(y):
    print 'Fold'
    k = 70 #num neighbors
    Xtrain = X[trainidx, :]  # using numpy's smart indexing
    print 'xtrained'
    Xtest = X[testidx, :]
    print 'xtested'
    ytrain = y[trainidx]
    print 'ytrained'
    ytest = y[testidx]
    print 'ytested'
    
    KNeighbors_model = KNeighborsClassifier(n_neighbors=k)  
    print 'model created'
    KNeighbors_model.fit(Xtrain, ytrain)
    print 'model fit'
    KNN_XtestP = KNeighbors_model.predict(Xtest)
    print 'model predicted'
    print k, 'neighbors' 
    print "KNN Score: ", KNeighbors_model.score(Xtest, ytest)
   # print 'KNN f1 score', f1_score(KNN_XtestP, ytest),'\n'


